# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [88]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [89]:
dataset=pd.read_csv("train.csv")[:1000]
dataset.drop(["id"],axis=1)

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
995,0,are you really or just pretending to be? #t...
996,0,i finally found a way how to delete old tweets...
997,0,@user what a #view #oregon #poland #usa #un...
998,0,"have you put in your #vote yet? if you havent,..."


In [90]:
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [91]:
# def clean_text(text):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     text = str(text).lower()
#     text = re.sub('\[.*?\]', ' ', text)
#     text = re.sub('https?://\S+|www\.\S+', ' ', text)
#     text = re.sub('<.*?>+', ' ', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
#     text = re.sub('\n', ' ', text)
#     text = re.sub('\w*\d\w*', ' ', text)
#     return text

In [92]:
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [93]:
dataset

,id,label,tweet
0,1,0,user when a father is dysfunctional and is so...
1,2,0,user user thanks for lyft credit i cant use ca...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation
...,...,...,...
995,996,0,are you really or just pretending to be tru...
996,997,0,i finally found a way how to delete old tweets...
997,998,0,user what a view oregon poland usa unitedst...
998,999,0,have you put in your vote yet if you havent do...


In [94]:
# Stemming
st=PorterStemmer()
dataset["tweet"]=dataset['tweet'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [95]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

,id,label,tweet
0,1,0,user when a father is dysfunct and is so selfi...
1,2,0,user user thank for lyft credit i cant use cau...
2,3,0,bihday your majesti
3,4,0,model i love u take with u all the time in ur
4,5,0,factsguid societi now motiv
...,...,...,...
995,996,0,are you realli or just pretend to be truthseek...
996,997,0,i final found a way how to delet old tweet you...
997,998,0,user what a view oregon poland usa unitedst ho...
998,999,0,have you put in your vote yet if you havent do...


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [96]:
max_words = 2000
max_len=50

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, padding='post', maxlen=max_len)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 user when a father is dysfunct and is so selfish he drag hi kid into hi dysfunct run
After Tokenization & Padding 
 [   1   48    4   44   11  709    8   11   24  517  103 1140  128  247
  201  128  709  339    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [97]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_words, embed_dim,input_length = _x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 50, 128)           256000    
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 50, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_7 (LSTM)               (None, 196)               254800    
                                                                 
 dense_7 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [98]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(800, 50) (800, 2)
(200, 50) (200, 2)


In [99]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
25/25 - 6s - loss: 0.3215 - accuracy: 0.9087 - 6s/epoch - 251ms/step
Epoch 2/7
25/25 - 2s - loss: 0.2644 - accuracy: 0.9275 - 2s/epoch - 89ms/step
Epoch 3/7
25/25 - 2s - loss: 0.2760 - accuracy: 0.9275 - 2s/epoch - 87ms/step
Epoch 4/7
25/25 - 2s - loss: 0.2688 - accuracy: 0.9275 - 2s/epoch - 89ms/step
Epoch 5/7
25/25 - 2s - loss: 0.2625 - accuracy: 0.9275 - 2s/epoch - 87ms/step
Epoch 6/7
25/25 - 2s - loss: 0.2686 - accuracy: 0.9275 - 2s/epoch - 88ms/step
Epoch 7/7
25/25 - 2s - loss: 0.2653 - accuracy: 0.9275 - 2s/epoch - 89ms/step


In [ ]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")